In [24]:
## METHOD 1: convert dictionary to spark dataframe and append to initialized sdf
# built-in imports
import re
from json import dump, load
from tqdm import tqdm
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import pandas as pd  
import os
# Import Spark modules
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Domain Scraper") \
    .getOrCreate()

#### create a spark data frame

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2)  # Update this to your liking

#Scrape suburb from the address
def extract_suburb(address: str) -> str:
    """Extract the suburb name from the property address."""
    match = re.search(r'(?<=, )\w+', address)
    if match:
        return match.group(0)
    return "Unknown"


def start_scrape() -> None:
    """ Function that scrapes https://www.domain.com.au and outputs the data into a json file

    Parameters:
    None

    Returns:
    None
    """
    schema = StructType([
    StructField("url", StringType(), True),
    StructField("name", StringType(), True),
    StructField("cost_text", StringType(), True),
    StructField("rooms", StringType(), True),
    StructField("desc", StringType(), True),
    StructField("parking", StringType(), True),
    StructField("street", StringType(), True),
    StructField("suburb", StringType(), True),
    StructField("postcode", StringType(), True),
    StructField("propertyType", StringType(), True),
    StructField("school", StringType(), True),
    StructField("features", StringType(), True),
    ])

    # begin code
    url_links = []
    property_metadata = defaultdict(dict)
    sdf = spark.createDataFrame([],schema)
    

    # generate list of urls to visit
    for page in N_PAGES:
        url = BASE_URL + f"/rent/?excludedeposittaken=1&state=vic&page={page}"
        print(f"Visiting {url}")
        bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "lxml")

        # find the unordered list (ul) elements which are the results, then
        # find all href (a) tags that are from the base_url website.
        index_links = bs_object.find("ul", {"data-testid": "results"}).findAll(
            "a", href=re.compile(f"{BASE_URL}/*")  # the `*` denotes wildcard any
        )

        for link in index_links:
            # if it's a property address, add it to the list
            if 'address' in link.get('class', []):
                url_links.append(link['href'])

    # for each url, scrape some basic metadata
    pbar = tqdm(url_links)
    success_count, total_count = 0, 0

    for property_url in pbar:
        try:
            bs_object = BeautifulSoup(urlopen(Request(property_url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "lxml")
            total_count += 1

            property_page = urlopen(Request(property_url, headers={'User-Agent': "PostmanRuntime/7.6.0"}))
            property_soup = BeautifulSoup(property_page, "lxml")

            # looks for the header class to get property name
            property_metadata[property_url]['name'] = bs_object.find("h1", {"class": "css-164r41r"}).text.strip()

            # looks for the div containing a summary title for cost
            property_metadata[property_url]['cost_text'] = bs_object.find(
                "div", {"data-testid": "listing-details__summary-title"}
            ).text.strip()


            # get rooms and parking
            rooms = bs_object.find("div", {"data-testid": "property-features"}).findAll(
                "span", {"data-testid": "property-features-text-container"}
            )

            # rooms
            property_metadata[property_url]['rooms'] = ", ".join(
                [re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'Bed' in feature.text or 'Bath' in feature.text]
            )

            # parking
            property_metadata[property_url]['parking'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'Parking' in feature.text]
            )

            # desc
            property_metadata[property_url]['desc'] = ", ".join(
                [re.findall(r'\d+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'desc' in feature.text]
            )
            
            # listingID:
            property_metadata[property_url]['listingid'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'listingId' in feature.text]
            )

            # street:
            property_metadata[property_url]['street'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'street' in feature.text]
            )

            # suburb:
            property_metadata[property_url]['suburb'] = extract_suburb(property_metadata[property_url]['name'])

            
            # postcode:
            property_metadata[property_url]['postcode'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'postcode' in feature.text]
            )

            # property type:
            property_metadata[property_url]['propertyType'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'apartment' in feature.text 
                 or 'unit' in feature.text or 'house' in feature.text or 'flat' in feature.text]
            )

            # schools:
            property_metadata[property_url]['school'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'school' in feature.text]
            )

            # features:
            property_metadata[property_url]['features'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'feature' in feature.text]
            )

            # loanfinder:
            property_metadata[property_url]['loan'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'loan' in feature.text]
            )

            # listingSummary:
            property_metadata[property_url]['listingsummary'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'summary' in feature.text]
            )

            # suburb insights:
            property_metadata[property_url]['suburbInsights'] = ", ".join(
                [re.findall(r'\S+\s[A-Za-z]+', feature.text)[0] for feature in rooms if 'suburbInsights' in feature.text]
            )

            # property description
            property_metadata[property_url]['desc'] = bs_object.find("p").text.strip() if bs_object.find("p") else "N/A"


            # Scrape property description
            property_metadata[property_url]['desc'] = re.sub(r'<br\/>', '\n', str(property_soup.find("p"))).strip('</p>')
           
            """
            # Write each row to the CSV
            writer.writerow([
                property_url,
                property_metadata[property_url]['name'],
                property_metadata[property_url]['cost_text'],
                property_metadata[property_url]['rooms'],
                property_metadata[property_url]['parking'],
                property_metadata[property_url]['desc'],
                property_metadata[property_url]['listingid'],
                property_metadata[property_url]['street'],
                property_metadata[property_url]['suburb'],
                property_metadata[property_url]['postcode'],
                property_metadata[property_url]['propertyType'],
                property_metadata[property_url]['school'],
                property_metadata[property_url]['features'],
                property_metadata[property_url]['loan'],
                property_metadata[property_url]['listingsummary'],
                property_metadata[property_url]['suburbInsights']
            ])
            """
            success_count += 1
            temp_sdf = spark.createDataFrame(property_metadata)
            sdf.union(temp_sdf)

        except AttributeError:
            print(f"Issue with {property_url}")

        pbar.set_description(f"{(success_count / total_count * 100):.0f}% successful")

        # output to example json in data/raw/
    with open('../data/raw/example.json', 'w') as f:
        dump(property_metadata, f)

def convert_to_parquet(filepath: str, output_path: str) -> None:
    """ Function converts a json file into a parquet file

    Parameters:
    filepath (str): the filepath that locates our json data

    output_path (str): the filepath that we will place our new parquet file into

    Returns:
    None
    """
    with open(filepath) as f:
        data = load(f)

    new_data = change_json_format(data)

    # conversion from json -> dataframe -> parquet
    df = pd.DataFrame(new_data)
    df.to_parquet(output_path, engine='pyarrow')

    delete_json_file(filepath)

# function that changes the formatting of the json file
def change_json_format(data: dict) -> dict:
    """ Function grabs the renames the json keys to the words after the last backslash in the url and adds the url as an item

    Parameters:
    data (dict): json dictionary we are changing

    Returns:
    dict: our new json dictionary
    
    """
    new_data = {}
    for i in data.keys():
        new_name = i.rsplit('/', 1)[-1]
        new_data[new_name] = data[i]
        new_data[new_name]["href"] = i
    return new_data

def delete_json_file(filepath: str) -> None:
    """ Function deletes the json file we are converting from

    Parameters:
    filepath (string): filepath to the json file we are deleting

    Returns:
    None
    """
    try:
        os.remove(filepath)
        print(f"File '{filepath}' deleted successfully")
    except FileNotFoundError:
        print(f"File '{filepath}' not found")
    except PermissionError:
        print(f"Permission denied: '{filepath}'")
    except Exception as e:
        print(f"An error occurred: {e}")

24/09/11 11:24:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [25]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

from pyarrow import json
import pyarrow.parquet as pq

24/09/11 11:24:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [26]:
def convert_to_parquet(filepath: str, output_path: str) -> None:
    """ Function converts a JSON file into a parquet file """
    with open(filepath) as f:
        data = load(f)

    new_data = change_json_format(data)

    # Conversion from JSON -> DataFrame -> Parquet
    df = pd.DataFrame(new_data)
    df.to_parquet(output_path, engine='pyarrow')

    delete_json_file(filepath)

def change_json_format(data: dict) -> dict:
    """ Function renames JSON keys and adds the URL as an item """
    new_data = {}
    for i in data.keys():
        new_name = i.rsplit('/', 1)[-1]
        new_data[new_name] = data[i]
        new_data[new_name]["href"] = i
    return new_data

def delete_json_file(filepath: str) -> None:
    """ Function deletes the JSON file """
    try:
        os.remove(filepath)
        print(f"File '{filepath}' deleted successfully")
    except FileNotFoundError:
        print(f"File '{filepath}' not found")
    except PermissionError:
        print(f"Permission denied: '{filepath}'")
    except Exception as e:
        print(f"An error occurred: {e}")

In [43]:
#Working METHOD
import re
from json import dump
from tqdm import tqdm
from collections import defaultdict
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

# Initialize Spark session
spark = SparkSession.builder.appName("PropertyScraper").getOrCreate()

# Constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 2)  # Update this to your liking

# Load suburbs CSV
suburbs_df = pd.read_csv('postcodes.csv')  # Ensure this CSV contains 'suburb' and 'postcode' columns

def start_scrape():
    """Function that scrapes https://www.domain.com.au and outputs the data into a JSON file"""

    # Define schema for the Spark DataFrame
    schema = StructType([
        StructField("url", StringType(), True),
        StructField("postcode", StringType(), True),
        StructField("suburb", StringType(), True),
        StructField("name", StringType(), True),
        StructField("cost_text", StringType(), True),
        StructField("beds", StringType(), True),  # Separate field for beds
        StructField("baths", StringType(), True),  # Separate field for baths
        StructField("parking", StringType(), True),  # Parking field
        StructField("property_type", StringType(), True),  # Parking field
    ])


    # Initialize an empty DataFrame with the schema
    property_metadata = spark.createDataFrame([], schema)

    # Loop through each suburb and its postcode
    for index, row in suburbs_df.iterrows():
        suburb = row['locality'].lower().replace(' ', '-')  # Convert to lowercase and hyphenate
        postcode = row['postcode']

        print(f"Scraping data for {suburb} ({postcode})")

        url_links = []

        # Generate list of URLs to visit
        for page in N_PAGES:
            url = BASE_URL + f"/rent/{suburb}-vic-{postcode}/?ssubs=0&sort=suburb-asc&page={page}"
            try:
                bs_object = BeautifulSoup(urlopen(Request(url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "lxml")

                # Find property links
                index_links = bs_object.find("ul", {"data-testid": "results"}).findAll(
                    "a", href=re.compile(f"{BASE_URL}/*")
                )

                for link in index_links:
                    # If it's a property address, add it to the list
                    if 'address' in link.get('class', []):
                        url_links.append(link['href'])

            except Exception as e:
                print(f"Error fetching {url}: {e}")

        # For each URL, scrape some basic metadata
        pbar = tqdm(url_links)
        success_count, total_count = 0, 0

        for property_url in pbar:
            try:
                bs_object = BeautifulSoup(urlopen(Request(property_url, headers={'User-Agent': "PostmanRuntime/7.6.0"})), "lxml")
                total_count += 1

                # Get property name
                name = bs_object.find("h1", {"class": "css-164r41r"}).text.strip()


                # Get cost text
                cost_text = bs_object.find("div", {"data-testid": "listing-details__summary-title"}).text.strip()

                # Get rooms (beds and baths)
                rooms = bs_object.find("div", {"data-testid": "property-features"}).findAll(
                    "span", {"data-testid": "property-features-text-container"}
                )

                # Initialize variables
                beds, baths, parking = None, None, '0'  # Default value for parking is '0 Car'

                for feature in rooms:
                    text = feature.text
                    if 'Bed' in text:
                        beds_match = re.findall(r'\d+', text)
                        if beds_match:
                            beds = beds_match[0]  # Extract the number of beds
                    elif 'Bath' in text:
                        baths_match = re.findall(r'\d+', text)
                        if baths_match:
                            baths = baths_match[0]  # Extract the number of baths
                    elif 'Car' in text or 'Parking' in text:
                        parking_match = re.findall(r'\d+', text)
                        if parking_match:
                            parking = parking_match[0]  # Extract the number of parking spaces

                property_type_container = bs_object.find("div", {"data-testid": "listing-summary-property-type"})
                property_type = property_type_container.get_text(strip=True)

                # Create a row and append it to the DataFrame
                row = [(property_url, postcode, suburb, name, cost_text, beds, baths, parking, property_type)]
                row_df = spark.createDataFrame(row, schema)
                property_metadata = property_metadata.union(row_df)
                success_count += 1

            except AttributeError:
                print(f"Error scraping {property_url}: missing data")

            pbar.set_description(f"{(success_count / total_count * 100):.0f}% successful")

        # Show the DataFrame to ensure data is being appended
        property_metadata.show()

    # Output to JSON file
    property_metadata.write.json('../data/raw/work.json', mode='overwrite')

# Start scraping
start_scrape()


Scraping data for melbourne (3000)


100% successful: 100%|██████████| 20/20 [00:14<00:00,  1.39it/s]


+--------------------+--------+---------+--------------------+--------------+----+-----+-------+--------------------+
|                 url|postcode|   suburb|                name|     cost_text|beds|baths|parking|       property_type|
+--------------------+--------+---------+--------------------+--------------+----+-----+-------+--------------------+
|https://www.domai...|    3000|melbourne|1503/270 King Str...| $850 Per Week|   4|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|26/418 St Kilda R...|       $725 pw|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|3712/80 ABeckett ...| $675 per week|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|3002B/11 Rose Lan...| $650 per week|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|103/300 Swanston ...|       $620 pw|   1|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|201/23 Queens R

0it [00:00, ?it/s]


+--------------------+--------+---------+--------------------+--------------+----+-----+-------+--------------------+
|                 url|postcode|   suburb|                name|     cost_text|beds|baths|parking|       property_type|
+--------------------+--------+---------+--------------------+--------------+----+-----+-------+--------------------+
|https://www.domai...|    3000|melbourne|1503/270 King Str...| $850 Per Week|   4|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|26/418 St Kilda R...|       $725 pw|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|3712/80 ABeckett ...| $675 per week|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|3002B/11 Rose Lan...| $650 per week|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|103/300 Swanston ...|       $620 pw|   1|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|201/23 Queens R

100% successful: 100%|██████████| 20/20 [00:13<00:00,  1.49it/s]


+--------------------+--------+---------+--------------------+--------------+----+-----+-------+--------------------+
|                 url|postcode|   suburb|                name|     cost_text|beds|baths|parking|       property_type|
+--------------------+--------+---------+--------------------+--------------+----+-----+-------+--------------------+
|https://www.domai...|    3000|melbourne|1503/270 King Str...| $850 Per Week|   4|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|26/418 St Kilda R...|       $725 pw|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|3712/80 ABeckett ...| $675 per week|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|3002B/11 Rose Lan...| $650 per week|   2|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|103/300 Swanston ...|       $620 pw|   1|    1|      1|Apartment / Unit ...|
|https://www.domai...|    3000|melbourne|201/23 Queens R

100% successful:  20%|██        | 4/20 [00:02<00:08,  1.98it/s]


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import os

# Define the folder path containing JSON files
folder_path = '../data/raw/work.json'

# List all files in the directory
json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]

# Initialize an empty list to store DataFrames
dataframes = []

# Read each JSON file into a DataFrame
for file in json_files:
    file_path = os.path.join(folder_path, file)
    # Read JSON file
    df = pd.read_json(file_path, lines=True)
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print(combined_df.head(12))


                                                  url  \
0   https://www.domain.com.au/4-45-ormond-esplanad...   
1   https://www.domain.com.au/506-6-percy-place-pr...   
2   https://www.domain.com.au/414-25-johnston-stre...   
3   https://www.domain.com.au/warrnambool-vic-3280...   
4   https://www.domain.com.au/103-300-swanston-str...   
5   https://www.domain.com.au/89-axford-boulevard-...   
6   https://www.domain.com.au/1403-325-collins-str...   
7   https://www.domain.com.au/39-foundry-circuit-b...   
8   https://www.domain.com.au/421-528-swanston-str...   
9   https://www.domain.com.au/401-3-5-union-street...   
10  https://www.domain.com.au/2-125-bell-street-iv...   
11  https://www.domain.com.au/1-7-gordon-street-to...   

                                            name                  cost_text  \
0         4/45 Ormond Esplanade, Elwood VIC 3184                $670 weekly   
1            506/6 Percy Place, Prahran VIC 3181                    $720.00   
2   414/25 Johnston S